In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
df = session.table("SVI.PUBLIC.SVI_HOSPITAL_MERGED").to_pandas()
df.columns.tolist()

In [ ]:
# Label
y = df["IS_MEDICAL_DESERT"].astype(int)

# Features (drop label + county)
X = df.drop(columns=["IS_MEDICAL_DESERT", "county"])

X.head(), y.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np

log_model = LogisticRegression(max_iter=500)
log_model.fit(X_train_scaled, y_train)

coef_table = pd.DataFrame({
    "feature": X.columns,
    "coefficient": log_model.coef_[0]
})

coef_table["odds_ratio"] = np.exp(coef_table["coefficient"])

coef_table.sort_values("coefficient", ascending=False)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd

vif = pd.DataFrame()
vif["feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif

In [ ]:
import matplotlib.pyplot as plt

# Sort coefficients from most negative → most positive
coef_plot = coef_table.sort_values("coefficient", ascending=True)

plt.figure(figsize=(10, 8))

plt.barh(
    coef_plot["feature"],
    coef_plot["coefficient"],
    color="steelblue",
    edgecolor="black"
)

# Clear, specific title
plt.title("Logistic Regression Coefficients for Predicting Medical Desert Status", fontsize=14)

# Clear x-axis meaning
plt.xlabel("Coefficient Value (Standardized Effect Size)", fontsize=12)

# Clear y-axis meaning
plt.ylabel("Predictor Variables", fontsize=12)

plt.grid(axis="x", linestyle="--", alpha=0.4)

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Base decision tree (no tuning yet)
tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)

tree

In [ ]:
from sklearn.model_selection import GridSearchCV

# Parameter grid for tuning
params = {
    "max_depth": [3, 5, 7, 10, None],
    "min_samples_split": [2, 5, 10, 20]
}

# Base model
base_tree = DecisionTreeClassifier(random_state=42)

# Grid search
grid = GridSearchCV(
    estimator=base_tree,
    param_grid=params,
    cv=5,
    scoring="accuracy"
)

grid.fit(X_train, y_train)

# Best tuned model
best_tree = grid.best_estimator_
best_tree

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 12))
plot_tree(
    best_tree,
    feature_names=X.columns,
    class_names=["Not Desert", "Desert"],
    filled=True,
    rounded=True,
    fontsize=10
)
plt.title("Decision Tree for Predicting Medical Desert Status")
plt.show()

In [ ]:
from sklearn.tree import _tree

tree_ = best_tree.tree_
feature_names = X.columns

rules = []

def recurse(node, depth, prefix=""):
    if tree_.feature[node] != _tree.TREE_UNDEFINED:
        feature = feature_names[tree_.feature[node]]
        threshold = tree_.threshold[node]

        rules.append(f"If {feature} <= {threshold:.3f}")
        recurse(tree_.children_left[node], depth + 1, prefix + "  ")

        rules.append(f"If {feature} > {threshold:.3f}")
        recurse(tree_.children_right[node], depth + 1, prefix + "  ")
    else:
        value = tree_.value[node]
        prediction = value.argmax()
        rules.append(f"→ Predict: {'Desert' if prediction == 1 else 'Not Desert'}")

# Start at root
recurse(0, 1)

for r in rules:
    print(r)